In [57]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

In [15]:
query = """
CALL gds.graph.project(
    'got1',
    'Person',
    {
        INTERACT: { properties : 'weight'}
    },
    {nodeProperties:'seed'}
    )
YIELD
  graphName AS graph,
  relationshipProjection AS knowsProjection,
  nodeCount AS nodes,
  relationshipCount AS rels
"""
graph.run(query)

graph,knowsProjection,nodes,rels
got1,"{INTERACT: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACT', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",399,2433


In [16]:
query = """
CALL gds.pageRank.stream('got1',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""

graph.run(query)


page,score
TYRION,7.2829717554208235
VARYS,5.674118882216431
WALDER,4.602201092362613


In [17]:
query = """
CALL gds.louvain.write.estimate('got1', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
399,2433,30881,634216,[30 KiB ... 619 KiB]


In [20]:
graph.run("CALL gds.louvain.stats('got1')YIELD communityCount")


communityCount
23


In [13]:
query = """
CALL gds.louvain.stream('got1',{ seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label 
LIMIT 5
"""

graph.run(query)

label,communityId
Addam,151
Adrack Humble,94
Aegon,51


In [64]:
query = """
CALL gds.graph.drop('got1') YIELD graphName;
"""
graph.run(query)

ClientError: [Procedure.ProcedureCallFailed] Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `got1` does not exist on database `neo4j`. It might exist on another database.

# Season by Season Analysis

## Season  1

In [65]:
query = """ 
CALL gds.graph.project.cypher(
  'got1',
  'MATCH (n:Person)-[r:INTERACT {season: 1}]-(m:Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACT {season: 1}]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""

graph.run(query)


graph,nodeQuery,nodes,relationshipQuery,rels
got1,"MATCH (n:Person)-[r:INTERACT {season: 1}]-(m:Person) RETURN id(n) AS id, n.seed AS seed",116,"MATCH (n:Person)-[r:INTERACT {season: 1}]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",582


In [66]:
# The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('got1', { writeProperty: 'community_S1' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)


nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
116,582,12729,580952,[12729 Bytes ... 567 KiB]


In [67]:
# The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('got1', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)


label,communityId
Addam,117
Aegon,97
Aemon,97


In [68]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('got1')
YIELD communityCount
"""
graph.run(query)


communityCount
7


In [69]:
# Degree
query = """
CALL gds.degree.stream('got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)


name,score
NED,30.0
TYRION,19.0
CATELYN,18.0


In [70]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'got1',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)


name,score
NED,299.0
TYRION,174.0
CATELYN,163.0


In [29]:
# Pagerank
query = """
CALL gds.pageRank.stream('got1',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)


page,score
TYRION,2.966833084860784
YOREN,2.7214507949389377
TYSHA,1.4430250415430863
